In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm.notebook')

import transformers
from tqdm import tqdm
import torch
import json
import ast
import numpy as np
import pandas as pd

from utilities import promptGenerator

from databench_eval import Evaluator
from databench_eval.utils import load_qa, load_table

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
device = torch.device("cuda:3")  # Select GPU 3
torch.cuda.set_device(device)
#model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_name = "google/gemma-3-4b-it"

In [3]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.10it/s]
Device set to use cuda:0


In [4]:
qa_dev = load_qa(lang="ES", name="iberlef", split="dev")

In [5]:
def build_full_prompt(prompt, type_prompt):
    if type_prompt == "code":
        role = {"role": "system", "content": "You are a numpy and pandas code generator. Your goal is to complete the function provided."}
    elif type_prompt == "instructions":
        role = {"role": "system", "content": "You are an experienced software engineer. Your objective is to describe the high-level steps or procedures necessary to address and solve a given question."}
    elif type_prompt == "columns":
        role = {"role": "system", "content": "You are an experienced data analyst. Your objective is to select the column(s) of a DataFrame needed to answer a given question."}
    elif type_prompt == "code_correction":
        role = {"role": "system", "content": "You are an experienced Python developer. Your objective is to fix the following bug."}
    else:
        role = {"role": "system", "content": "You are a helpful assistant, please follow every instruction in the following prompt."}
    full_prompt = [
        role,
        {"role": "user", "content": prompt}
    ]
    return full_prompt

In [6]:
def example_postprocess(response: str, row: dict):
    try:
        df = load_table(row["dataset"], lang="ES")
        
        global ans
        lead = """
def answer(df):
    """
        exec_string = (
            response
            + "\nans = answer(df)"
        )
        local_vars = {"df": df, "pd": pd, "np": np}
        exec(exec_string, local_vars)

        ans = local_vars["ans"]
        if isinstance(ans, pd.Series):
            ans = ans.tolist()
        elif isinstance(ans, pd.DataFrame):
            ans = ans.iloc[:, 0].tolist()
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [7]:
def save_responses(responses, save_path: str) -> None:
    with open(save_path, "w") as f:
        for response in responses:
            f.write(str(response).replace("\n", " ") + "\n")

In [8]:
responses = []
it = 1

for instance in tqdm(qa_dev):
    prompt_handler = promptGenerator.promptGenerator(instance, True)
    print(instance["question"])

    full_prompt_columns = build_full_prompt(prompt_handler.getColumnsV2(), "default")

    outputs_columns = pipeline(
        full_prompt_columns,
        max_new_tokens=200,
        pad_token_id=pipeline.tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.8
    )

    columns = outputs_columns[0]["generated_text"][-1]['content']
    prompt_handler.setReducedColumns(columns)
    print(columns)
    
    
    full_prompt_instructions = build_full_prompt(prompt_handler.getInstructions(columns), "default")
    
    outputs_instructions = pipeline(
        full_prompt_instructions,
        max_new_tokens=600,
        pad_token_id=pipeline.tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.8
    )

    cot_steps = outputs_instructions[0]["generated_text"][-1]['content']
    print(cot_steps)
    

    full_prompt_code = build_full_prompt(prompt_handler.getCode(cot_steps, columns), "default")
    
    outputs_code = pipeline(
        full_prompt_code,
        max_new_tokens=800,
        pad_token_id=pipeline.tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.8
    )

    code = outputs_code[0]["generated_text"][-1]['content']
    print(code)
    
    output_code = example_postprocess(code, instance)

    if str(output_code).startswith("__CODE_ERROR__"):
        print("---ERROR---")
        attempts = 1
        while str(output_code).startswith("__CODE_ERROR__") and attempts <= 3:
            full_prompt_correct_code = build_full_prompt(prompt_handler.getCorrectCode(cot_steps, columns, code, output_code), "default")
            outputs_code = pipeline(
                full_prompt_correct_code,
                max_new_tokens=800,
                pad_token_id=pipeline.tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.6,
                top_p=0.8
            )
        
            code = outputs_code[0]["generated_text"][-1]['content']
            #print(code)

            output_code = example_postprocess(code, instance)
            attempts += 1

    responses.append([str(output_code)])

    if(it % 2 == 0):
        evaluator = Evaluator(qa=qa_dev)
        print(f"DataBenchSPA accuracy is {evaluator.eval(responses)}")
    #    #break
    it += 1

  0%|                                                                                                                                                                | 0/100 [00:00<?, ?it/s]

¿Hay más mujeres que hombres en esta encuesta?


skipping cudagraphs due to skipping cudagraphs due to multiple devices: device(type='cuda', index=1), device(type='cuda', index=2), device(type='cuda', index=3), device(type='cuda', index=0)


['Sexo', '¿Cuántos hijos_as tienes o crees que vas a tener a lo largo de tu vida?']
1.  Identify the column containing gender information.
2.  Count the occurrences of each gender.
3.  Compare the counts to determine if there are more women than men.
import pandas as pd
import numpy as np

def answer(df: pd.DataFrame):
    """Explain each step with comments above."""
    # Check if the DataFrame is empty
    if df.empty:
        return True  # If empty, assume more women than men

    # Identify the column containing gender information
    gender_column = 'Sexo'

    # Count the occurrences of each gender
    gender_counts = df[gender_column].value_counts()

    # Get the counts for 'Mujer' and 'Hombre'
    women_count = gender_counts.get('Mujer', 0)
    men_count = gender_counts.get('Hombre', 0)

    # Compare the counts to determine if there are more women than men
    return women_count > men_count  # Return True if women are more numerous


  1%|█▌                                                                                                                                                    | 1/100 [00:37<1:01:37, 37.34s/it]

...

DataBenchSPA accuracy is 0.3
¿Cuál es el tamaño de hábitat más común?
['Tamaño de hábitat']
1.  Identify the column containing habitat size information.
2.  Determine the frequency of each unique habitat size.
3.  Identify the most frequent habitat size.
4.  Express the most frequent habitat size as the answer.
def answer(df: pd.DataFrame):
    """Explain each step with comments above."""
    # Identify the column containing habitat size information.
    habitat_column = 'Tamaño de hábitat'
    
    # Determine the frequency of each unique habitat size.
    habitat_counts = df[habitat_column].value_counts()
    
    # Identify the most frequent habitat size.
    most_frequent_habitat = habitat_counts.idxmax()
    
    # Express the most frequent habitat size as the answer.
    return most_frequent_habitat



 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 99/100 [39:18<00:20, 20.69s/it]

¿Cuál es el sexo más común en la encuesta?
['Sexo']
1.  Identify the SEX column.
2.  Determine the most frequent value within the SEX column.
3.  Express the most frequent value as the answer.
def answer(df: pd.DataFrame):
    """Explain each step with comments above."""
    # Identify the SEX column
    sex_column = 'Sexo'

    # Determine the most frequent value within the SEX column
    sex_counts = df[sex_column].value_counts()

    # Express the most frequent value as the answer
    most_frequent_sex = sex_counts.idxmax()

    # Return the most frequent sex
    return most_frequent_sex



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [39:27<00:00, 23.67s/it]

DataBenchSPA accuracy is 0.32


In [9]:
evaluator = Evaluator(qa=qa_dev)
print(f"DataBenchSPA accuracy is {evaluator.eval(responses)}")
save_responses(responses, "predictions.txt")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 8371.53it/s]

DataBenchSPA accuracy is 0.32


In [10]:
responses

[['True'],
 ['__CODE_ERROR__: Can only use .str accessor with string values!'],
 ['False'],
 ['False'],
 ['0'],
 ['__CODE_ERROR__: Categorical is not ordered for operation min\nyou can use .as_ordered() to change the Categorical to an ordered one\n'],
 ['None'],
 ['True'],
 ['93'],
 ['2'],
 ['10'],
 ['__CODE_ERROR__: unterminated string literal (detected at line 7) (<string>, line 7)'],
 ['__CODE_ERROR__: Unordered Categoricals can only compare equality or not'],
 ["__CODE_ERROR__: 'numpy.int64' object has no attribute 'index'"],
 ['Cuenta ajena (p.e: empleado)'],
 ["['Alta / Media Alta', 'Media baja / Baja']"],
 ["['Alta / Media Alta', 'Media baja / Baja']"],
 ['__CODE_ERROR__: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()'],
 ["['PP (Partido Popular)', 'PSOE (Partido Socialista Obrero Español)', 'No lo sé']"],
 ["['PP (Partido Popular)', 'PSOE (Partido Socialista Obrero Español)', 'No lo sé']"],
 ['[1, 2, 3, 4]'],
 ['[2, 3, 1]'],
 ['[504,

In [11]:
qa_dev['answer'][:20]

['true',
 'false',
 'false',
 'true',
 '1268',
 '18',
 '93',
 ' false',
 '93',
 '2',
 '10',
 'Barcelona',
 'Pensionista',
 'Otros',
 'Cuenta ajena (p.e: empleado)',
 "['Alta / Media Alta', 'Media baja / Baja']",
 "['Alta / Media Alta', 'Media']",
 "['He tenido/se ha tenido que contraer deuda', 'Ahorro/se ahorra bastante dinero cada mes']",
 "['PP (Partido Popular)', 'PSOE (Partido Socialista Obrero Español)', 'Vox']",
 "['PP (Partido Popular)', 'PSOE (Partido Socialista Obrero Español)', 'No lo sé']"]